In [55]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd

fl = list(glob.glob('../features/*.pkl'))
for l in sorted(fl):
    print("'{}'".format(l))

'../features/cnn_1_aug1_feat.pkl'
'../features/cnn_1_aug_1_fold8_feat.pkl'
'../features/cnn_1_aug_rescale_preprocess_feat.pkl'
'../features/cnn_1_aug_skimage_denoise_feat.pkl'
'../features/cnn_1_aug_skimage_preprocess_feat.pkl'
'../features/cnn_1_feat.pkl'
'../features/cnn_2_aug1_feat.pkl'
'../features/cnn_2_aug_1_fold8_feat.pkl'
'../features/cnn_2_aug_denoise_preprocess_feat.pkl'
'../features/cnn_2_aug_rescale_preprocess_feat.pkl'
'../features/cnn_2_aug_skimage_preprocess_feat.pkl'
'../features/cnn_2_feat.pkl'
'../features/cnn_3_aug1_feat.pkl'
'../features/cnn_3_aug_1_fold8_feat.pkl'
'../features/cnn_3_aug_denoise_preprocess_feat.pkl'
'../features/cnn_3_aug_rescale_preprocess_feat.pkl'
'../features/cnn_3_aug_skimage_preprocess_feat.pkl'
'../features/cnn_4_aug1_feat.pkl'
'../features/cnn_4_aug1_feat_add_early.pkl'
'../features/cnn_4_aug_1_fold8_feat.pkl'
'../features/cnn_4_aug_denoise_preprocess_feat.pkl'
'../features/cnn_4_aug_rescale_preprocess_feat.pkl'
'../features/cnn_4_aug_skimag

In [56]:
def load_x_y():
    train_x,test_x=[],[]
    for f in fl:
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

(1604, 56) (8424, 56) (1604,)


In [57]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        params = {
                'colsample_bytree': 0.75,
                'colsample_bylevel':0.95,
                'gamma':0.2,
                'subsample': 0.9,
                'eta': 0.07,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight': 1.0
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(test_x)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 500, watchlist, 
                        early_stopping_rounds=30,
                        verbose_eval=1000)
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict(d_valid))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            w_test_pred = m.predict(d_test)*curr_r_val_loss
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred
        print(submission.head())
        submission.to_csv('../results/all_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=w_test_pred
        print(submission.head())
        submission.to_csv('../results/weighted_all_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)

    
print('def done')

def done


In [58]:
cv_test(3,42,True)

[0]	train-logloss:0.637037	valid-logloss:0.640462
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[131]	train-logloss:0.044452	valid-logloss:0.160958

[0]	train-logloss:0.637773	valid-logloss:0.640521
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[92]	train-logloss:0.067027	valid-logloss:0.164897

[0]	train-logloss:0.637336	valid-logloss:0.63899
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[82]	train-logloss:0.073132	valid-logloss:0.17032

local average valid loss 0.169251661052 val loss std 0.0059555521926
         id  is_iceberg
0  5941774d    0.013630
1  4023181e    0.943625
2  b20200e4    0.034760

In [59]:
cv_test(5,42,True)

[0]	train-logloss:0.637737	valid-logloss:0.641389
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[85]	train-logloss:0.075987	valid-logloss:0.159165

[0]	train-logloss:0.63772	valid-logloss:0.639149
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[103]	train-logloss:0.067981	valid-logloss:0.140546

[0]	train-logloss:0.63693	valid-logloss:0.638803
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[80]	train-logloss:0.073215	valid-logloss:0.19237

[0]	train-logloss:0.638264	valid-logloss:0.640832
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-lo

In [60]:
cv_test(10,42,True)

[0]	train-logloss:0.637348	valid-logloss:0.639638
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[104]	train-logloss:0.065473	valid-logloss:0.178839

[0]	train-logloss:0.63846	valid-logloss:0.639136
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[128]	train-logloss:0.05689	valid-logloss:0.145799

[0]	train-logloss:0.638586	valid-logloss:0.641752
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[222]	train-logloss:0.028512	valid-logloss:0.136012

[0]	train-logloss:0.638349	valid-logloss:0.637826
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid